In [222]:
import pandas as pd
import numpy as np
import csv

def csv_to_data_frame(file_name):
    titles = []
    datas = []

    with open(file_name, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                titles = row
                line_count += 1
            datas.append([int(value) if value.isnumeric() else value for (key, value) in row.items()])
            line_count += 1
        names = pd.DataFrame(datas, columns=titles)

        return names

In [223]:
data = csv_to_data_frame("NationalNames.csv")

In [50]:
# task_3
data.columns

Index(['Id', 'Name', 'Year', 'Gender', 'Count'], dtype='object')

In [51]:
#task_4
data[["Id", "Year", "Count"]].describe()

,Id,Year,Count
count,1.825433e+06,1.825433e+06,1.825433e+06
mean,9.127170e+05,1.972620e+03,1.846879e+02
std,5.269573e+05,3.352891e+01,1.566711e+03
min,1.000000e+00,1.880000e+03,5.000000e+00
25%,4.563590e+05,1.949000e+03,7.000000e+00
50%,9.127170e+05,1.982000e+03,1.200000e+01
75%,1.369075e+06,2.001000e+03,3.200000e+01
max,1.825433e+06,2.014000e+03,9.968000e+04


In [53]:
#task_5
data["Name"].nunique()

93889

In [58]:
#task_8
max_item = data.loc[data['Count'].idxmax()]
f"The name is '{max_item['Name']}' in {max_item['Year']}"

"The name is 'Linda' in 1947"

In [63]:
#task_9
len(data.loc[data["Count"] == data["Count"].min()])

254615

In [115]:
#task_11
unique = data.groupby("Year")["Name"].nunique()
popular_year = unique.idxmax()
unique.loc[popular_year:popular_year]

Year
2008    32488
Name: Name, dtype: int64

In [121]:
#task_12
popular_group = data.groupby("Year").get_group(popular_year)
popular_group.loc[popular_group['Count'].idxmax()].loc["Name"]

'Jacob'

In [140]:
#task_13
group_name = data.groupby("Name")
jacob_group = group_name.get_group("Jacob")
jacob_f = jacob_group.loc[jacob_group["Gender"] == "F"]
f_index = jacob_f["Count"].idxmax()
jacob_f.loc[f_index:f_index]

,Id,Name,Year,Gender,Count
1455556,1455557,Jacob,2004,F,171


In [253]:
#task_14
group = pd.DataFrame(data.groupby(["Year", "Gender"])["Name"].unique())
year_set = list(set([item[0] for item in group.index]))

m_array = group.loc[(year_set[0], "M")].values[0]
f_array = group.loc[(year_set[0], "F")].values[0]
max_count = len(np.intersect1d(m_array, f_array))
max_year = year_set[0]
for i in range(1, len(year_set)):
    year = year_set[i]
    m_array = group.loc[(year, "M")].values[0]
    f_array = group.loc[(year, "F")].values[0]
    count = len(np.intersect1d(m_array, f_array))
    if count > max_count:
        max_count = count
        max_year = year

pd.DataFrame([[max_year, max_count]], columns=["Year", "Gender_neutral_names"])


,Year,Gender_neutral_names
0,2008,2557
